In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor

def hour_bins(hour):
    '''Function to divide Flight Times into Windows based on observed average fares'''
    if hour in ['00','01','02','03','04','05','06', '07']:
        return '00-07'
    elif hour in ['08','09']:
        return '08-09'
    elif hour in ['10','11','12','13','14','15','16','17','18']:
        return '10-18'
    elif hour in ['19','20','21','22','23']:
        return '19-23'
        
train_url = 'train.csv'
#Read Train and Test Data
train, test = pd.read_csv(train_url), pd.read_csv('test.csv')
#train, test = pd.read_csv('train.csv'), pd.read_csv('test.csv')

#Business and Economy are binary. Business having a higher fare. Replacing with 1, 0
train = train.replace(['Business', 'Economy'], [1, 0])
test = test.replace(['Business', 'Economy'], [1, 0])

#Create New Features 'Title' and 'Gender' from the 'Name'
train['Title'] = train.apply(lambda row: row[0].split(" ")[0], axis = 1)
train['Gender'] = train.apply(lambda row: row[0].split(" ")[1].split("G")[0], axis = 1)

test['Title'] = test.apply(lambda row: row[0].split(" ")[0], axis = 1)
test['Gender'] = test.apply(lambda row: row[0].split(" ")[1].split("G")[0], axis = 1)

#Male and Female form a binary class. Males paying a higher fare. Replacing with 1, 0
train = train.replace(['M', 'F'], [1, 0])
test = test.replace(['M', 'F'], [1, 0])

#Create feature 'Age' from the 'Date of Birth' and 'Booking Date'
train['Age'] = train.apply(lambda row: int(row[6].split("-")[0]) - int(row[1].split("-")[0]), axis = 1)                                         
test['Age'] = test.apply(lambda row: int(row[6].split("-")[0]) - int(row[1].split("-")[0]), axis = 1)

#Drop 'Date of Birth'
train.drop('Date of Birth', axis = 1, inplace = True)
test.drop('Date of Birth', axis = 1, inplace = True)

#Create a feature called 'From-To' which is a measure of distance b/w cities in some sense
train['From-To'] = train.apply(lambda row: row[1]+'-'+row[2], axis = 1)
test['From-To'] = test.apply(lambda row: row[1]+'-'+row[2], axis = 1)

#Create a feature 'Days to Journey' from 'Flight Date' and 'Booking Date'
train['Flight Date'] = pd.to_datetime(train['Flight Date'])
train['Booking Date'] = pd.to_datetime(train['Booking Date'])
train['Days to Journey'] = train.apply(lambda row: (row[3] - row[5]).days, axis = 1)

test['Flight Date'] = pd.to_datetime(test['Flight Date'])
test['Booking Date'] = pd.to_datetime(test['Booking Date'])
test['Days to Journey'] = test.apply(lambda row: (row[3] - row[5]).days, axis = 1)

#Retaining only the 'Hours' component of 'Flight Time'
train['Hours'] = train.apply(lambda row: row[4].split(":")[0] , axis = 1)
test['Hours'] = test.apply(lambda row: row[4].split(":")[0] , axis = 1)

#Dividing 'Hours' into windows based on average fares
train['Hour'] = train.apply(lambda row: hour_bins(row[13]), axis = 1)
test['Hour'] = test.apply(lambda row: hour_bins(row[12]), axis = 1)

# Drop 'Hours'
train.drop('Hours', axis = 1, inplace = True)
test.drop('Hours', axis = 1, inplace = True)

#Drop 'Flight Time'
train.drop('Flight Time', axis = 1, inplace = True)
test.drop('Flight Time', axis = 1, inplace = True)

#Divide people into age groups by creating Age-bins (span 10 years)
train['Age_bin'] = train.apply(lambda row: str(int(row[9]/10)*10) +'-'+str(int(row[9]/10)*10 + 9)\
                               , axis = 1)
test['Age_bin'] = test.apply(lambda row: str(int(row[8]/10)*10) +'-'+str(int(row[8]/10)*10 + 9)\
                               , axis = 1)

#Drop unnecessary features
train.drop(['Age','Flight Date', 'Booking Date', 'Name'], axis = 1, inplace = True)
test.drop(['Age','Flight Date', 'Booking Date', 'Name'], axis = 1, inplace = True)
 
train = train.astype({'From':'category', 'To':'category', 'Class':'category', 'Title':'category',\
                     'From-To':'category', 'Hour':'category', 'Age_bin':'category'})
test = test.astype({'From':'category', 'To':'category', 'Class':'category', 'Title':'category',\
                    'From-To':'category', 'Hour':'category', 'Age_bin':'category'})

#Prepare Data for Model
train_Y = train['Fare']
train_X = train.drop('Fare', axis = 1)
test_X = test

#Make Predictions
train_X, test_X = pd.get_dummies(train_X), pd.get_dummies(test_X)
fin_model = GradientBoostingRegressor(n_estimators=100, max_depth=8)
fin_model.fit(train_X, train_Y)
Predictions = fin_model.predict(test_X)

C:\Program Files\Anaconda3\envs\py35\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


SyntaxError: 'return' outside function (<ipython-input-1-26778b037423>, line 95)

In [2]:
Predictions.tolist()

[1978.7937609935327,
 3081.8523795424426,
 12197.78517784184,
 6148.557872161117,
 6209.683744601755,
 8495.049592116118,
 10410.118282300826,
 2751.1795540150874,
 5539.950196699982,
 4619.840088428919,
 10837.152038574595,
 4238.713025843155,
 2725.4784753092895,
 10121.366225504227,
 3348.7941141471447,
 4797.163368090537,
 6784.742754461783,
 11768.102100213539,
 7576.836890322836,
 15902.723308329298,
 3127.1109267547604,
 7676.538194766803,
 9840.73695986392,
 3423.222945708395,
 11297.14602053852,
 1845.9497856845412,
 4283.15742315943,
 4918.799433254623,
 20134.42351825894,
 12606.833665821785,
 5022.063483577725,
 4695.180795940116,
 7879.639608339345,
 4602.76601802411,
 15064.25557977694,
 5936.31998773403,
 5871.216063591044,
 4460.815874593816,
 5936.31998773403,
 16460.506016306124,
 3498.5227429118836,
 2331.641235624077,
 6219.820281321458,
 12292.921784988928,
 1858.1695442546459,
 6045.824747837706,
 4775.275293221349,
 15599.56123710192,
 9154.42660162325,
 6219.540